# Homework #12 for Spring2020 6.036 (MIT OLL)

Although I've done all the preceding hws, with HW12 I've switched to doing these assignments in Jupyter and uploading to GitHub so I can more easily review my work.

Much of the code is supplied by the class, and much of the homework is filling in specific lines of code.  That is to say, I don't own most of this code, but it is publically available and usable under standard non-commercial licensing

### Begin class-provided code and data:

In [10]:
import pdb
import numpy as np
import time
import pickle

genres = ['Western', 'Comedy', 'Children', 'Crime', 'Musical',
          'Adventure', 'Drama', 'Horror', 'War', 'Documentary',
          'Romance', 'Animation', 'Film-Noir', 'Sci-Fi', 'Mystery',
          'Fantasy', 'IMAX', 'Action', 'Thriller']

# Data is a list of (a, i, r) triples
ratings_small = \
[(0, 0, 5), (0, 1, 3), (0, 3, 1),
 (1, 0, 4), (1, 3, 1),
 (2, 0, 1), (2, 1, 1), (2, 3, 5),
 (3, 0, 1), (3, 3, 4),
 (4, 1, 1), (4, 2, 5), (4, 3, 4)]


In [11]:
def pred(data, x):
    (a, i, r) = data
    (u, b_u, v, b_v) = x
    return np.dot(u[a].T,v[i]) + b_u[a] + b_v[i]

# X : n x k
# Y : n
def ridge_analytic(X, Y, lam):
    (n, k) = X.shape
    xm = np.mean(X, axis = 0, keepdims = True)   # 1 x n
    ym = np.mean(Y)                              # 1 x 1
    Z = X - xm                                   # d x n
    T = Y - ym                                   # 1 x n
    th = np.linalg.solve(np.dot(Z.T, Z) + lam * np.identity(k), np.dot(Z.T, T))
    # th_0 account for the centering
    th_0 = (ym - np.dot(xm, th))                 # 1 x 1
    return th.reshape((k,1)), float(th_0)


# Begin Homework!

### 2) Some movies are more equal than others

In [12]:
# Example from lab handout
Z = np.array([[1], [1], [5], [1], [5], [5], [1]])
b_v = np.array([[3], [3], [3], [3], [3], [5], [1]])
B = np.array([[1, 10], [1, 10], [10, 1], [1, 10], [10, 1], [5, 5], [5, 5]])
# Solution with offsets, using ridge_analytic provided in code file
u_a, b_u_a = ridge_analytic(B, (Z - b_v), 1)
print('With offsets', u_a, b_u_a)
# Solution using previous model, with no offsets
u_a_no_b = np.dot(np.linalg.inv(np.dot(B.T, B) + 1 * np.identity(2)), np.dot(B.T, Z))
print('With no offsets', u_a_no_b)

With offsets [[ 0.22024566]
 [-0.22193986]] 0.007623888182955896
With no offsets [[0.50148126]
 [0.0562376 ]]


#### 2A

The most difficult part of this was remembering the b_v_i offset for "bad ratings from everyone".  I originally got 1.988 as my answer, which was rejected by the autograder before I remembered to add it back in.

In [13]:
llama = np.array([[10],[1]])
b_v_i = 1
print(u_a.T@llama + b_u_a + b_v_i,u_a_no_b.T @ llama)

[[2.98814062]] [[5.07105019]]


#### 2B and 2C

trivial now that I remember b_v_i

In [14]:
robot = np.array([[1],[10]])
b_v_i = 5
print(u_a.T@robot + b_u_a + b_v_i,u_a_no_b.T @ robot)
b_v_i = 3
print(u_a.T@robot + b_u_a + b_v_i,u_a_no_b.T @ robot)

[[3.00847099]] [[1.0638573]]
[[1.00847099]] [[1.0638573]]


### 3) Implementing recommender systems

In [15]:
# The SGD outer loop
def mf_sgd(data_train, data_validate, step_size_fn, k=2, lam=0.02, max_iter=100, verbose=False):
    # size of the problem
    ndata = len(data_train)
    n = max(d[0] for d in data_train)+1
    m = max(d[1] for d in data_train)+1
    # Distribute the lambda among the users and items
    lam_uv = lam/counts(data_train,0), lam/counts(data_train,1)
    # Initial guess at u, b_u, v, b_v (also b)
    x = ([np.random.normal(1/k, size=(k,1)) for j in range(n)],
         np.zeros(n),
         [np.random.normal(1/k, size=(k,1)) for j in range(m)],
         np.zeros(m))
    di = int(max_iter/10.)
    for i in range(max_iter):
        if i%di == 0 and verbose:
            print('i=', i, 'train rmse=', rmse(data_train, x),
                  'validate rmse', data_validate and rmse(data_validate, x))
        step = step_size_fn(i)
        j = np.random.randint(ndata)            # pick data item
        sgd_step(data_train[j], x, lam_uv, step) # modify x
    print('SGD result for k =', k, ': rmse train =', rmse(data_train, x), '; rmse validate =', rmse(data_validate, x))
    return x

#### 3.1) update_U

the major issue here seems to be translating the phrasing of the regression from the lab.  Go slow and methodically and things make sense.
Try to make impatient jumps of logic and nothing makes sense, so go step by step!

Don't miss this:  "Note that u and v are lists of column vectors (rows of U, V)."   ie they're not arrays!

In the lab, Ia is the set of all movies rated by person a.
Ba is constructed by taking V and removing every movie not in Ia.
Y is the (incomplete) matrix of all the ratings we have (dimension of n users x m movies)
Then Za is a column vector of the YAi elements for all i in Ia
Then call ridge analytic(B, (Z - b_v), lam)



In [90]:
#update_U_test()
def update_U(data, vs_from_u, x, k, lam):
    (u, b_u, v, b_v) = x
    
    # for each user
    for i in range(len(vs_from_u)):
        if not vs_from_u[i]: continue
        c = len(vs_from_u[i])
        #print("there are ", c, " tuples in user ", i, "'s records")
        indexes = []
        B = np.array([[] for ix in range(k)])
        score = np.zeros((c,1))
        # for every rating they've done
        for j in range(c):
            
            rating = vs_from_u[i][j]
            #print(rating)
            indexes.append(int(rating[0])) #construct Ia
            score[j,0] = rating[1]
            B = np.hstack((B,v[rating[0]]))
        Z = score
        B = B.T
        b_v_reshape = b_v[indexes].reshape((b_v[indexes].shape[0],1)) # had to reshape this because the online checker uses a different format than the offline test code
        #print(B.shape, Z.shape, b_v_reshape.shape)
        th, th0 = ridge_analytic(B,Z-b_v_reshape,lam)
        u[i] = th
        b_u[i] = th0
    x = (u,b_u,v,b_v)
    return x


## copied from homework solution since it's cleaner
def update_V(data, us_from_v, x, k, lam):
    (u, b_u, v, b_v) = x
    for i in range(len(v)):
        if not us_from_v[i]: continue
        V = np.hstack([u[a] for (a, _) in us_from_v[i]]).T
        y = np.array([r-b_u[a] for (a, r) in us_from_v[i]])
        v[i], b_v[i] = ridge_analytic(V, y, lam)
    return x

In [17]:

#Test case for update_U
def update_U_test():
  '''
  This is a test function provided to help you debug your implementation
  '''
  k = 2
  lam = 0.01
  
  vs_from_u = \
  ([[(0, 5), (1, 3), (3, 1)],
   [(0, 4), (3, 1)],
   [(0, 1), (1, 1), (3, 5)],
   [(0, 1), (3, 4)],
   [(1, 1), (2, 5), (3, 4)]])
  
  np.random.seed(0)
  
  first = []
  for i in range(5):
    first.append(np.random.rand(2, 1))
  second = np.zeros((5, 1))
  third = []
  for i in range(5):
    third.append(np.random.rand(2, 1))
  fourth = np.zeros((5, 1))
  x0 = (first, second, third, fourth)
  
  x_result = update_U(ratings_small, vs_from_u, x0, k, lam)
  
  x_list = [[u.tolist() for u in x_result[0]],
            x_result[1].tolist(),
            [v.tolist() for v in x_result[2]],
            x_result[3].tolist()]
  
  
  assert np.all(np.isclose(x_list[0], np.array([[[4.048442188078757], [-2.5000082235465526]],
                                                [[3.2715388359271054], [-1.2879317400952521]],
                                                [[-6.237522315961142], [-2.9639103597721355]],
                                                [[-3.2715388359271054], [1.2879317400952521]],
                                                [[-4.87111151185168], [-1.761023196019822]] ])))
  assert np.all(np.isclose(x_list[1], 
                           np.array([[3.043665230868208], [2.048616799474877], [7.462166369240114], [2.951383200525123], [5.487071919883842]])))
  assert np.all(np.isclose(x_list[2], np.array([[[0.7917250380826646], [0.5288949197529045]],
                                       [[0.5680445610939323], [0.925596638292661]],
                                       [[0.07103605819788694], [0.08712929970154071]],
                                       [[0.02021839744032572], [0.832619845547938]],
                                       [[0.7781567509498505], [0.8700121482468192]] ])))
  assert np.all(np.isclose(x_list[3], np.array([[0.0], [0.0], [0.0], [0.0], [0.0]])))
  print("Test passed!")

update_U_test()


Test passed!


In [87]:
# Simple validate case
mf_als(ratings_small, ratings_small,lam=0.01, max_iter=10, k=2)

ALS result for k = 2 : rmse train = [[0.00223025]] ; rmse validate = [[0.00223025]]


([array([[ 1.22319618],
         [-0.84002598]]),
  array([[ 0.81430847],
         [-0.74411966]]),
  array([[-1.23397271],
         [ 0.81225434]]),
  array([[-0.80628622],
         [ 0.73678888]]),
  array([[1.18339142],
         [4.3807127 ]])],
 array([ 3.58480402,  2.94624916,  2.3787652 ,  2.01506587, -0.13938456]),
 [array([[ 0.74896744],
         [-0.57491886]]),
  array([[0.54666151],
         [0.33433815]]),
  array([[0.],
         [0.]]),
  array([[-1.25625301],
         [ 1.27742317]])],
 array([ 0.01424939, -0.97352078,  5.13938456,  0.02929921]))

### 3.2 sgd_step

Plug and play the gradient formulas from https://openlearninglibrary.mit.edu/courses/course-v1:MITx+6.036+1T2019/courseware/Week12/recommender_systems/?child=last

In [19]:
## provided code

# The SGD outer loop
def mf_sgd(data_train, data_validate, step_size_fn, k=2, lam=0.02, max_iter=100, verbose=False):
    # size of the problem
    ndata = len(data_train)
    n = max(d[0] for d in data_train)+1
    m = max(d[1] for d in data_train)+1
    # Distribute the lambda among the users and items
    lam_uv = lam/counts(data_train,0), lam/counts(data_train,1)
    # Initial guess at u, b_u, v, b_v (also b)
    x = ([np.random.normal(1/k, size=(k,1)) for j in range(n)],
         np.zeros(n),
         [np.random.normal(1/k, size=(k,1)) for j in range(m)],
         np.zeros(m))
    di = int(max_iter/10.)
    for i in range(max_iter):
        if i%di == 0 and verbose:
            print('i=', i, 'train rmse=', rmse(data_train, x),
                  'validate rmse', data_validate and rmse(data_validate, x))
        step = step_size_fn(i)
        j = np.random.randint(ndata)            # pick data item
        sgd_step(data_train[j], x, lam_uv, step) # modify x
    print('SGD result for k =', k, ': rmse train =', rmse(data_train, x), '; rmse validate =', rmse(data_validate, x))
    return x


In [20]:
## my code

def sgd_step(data, x, lam, step):
    (a, i, r) = data
    (u, b_u, v, b_v) = x
    (lam_u, lam_v) = lam
    Y_ai = r
    v_i = v[i]
    u_a = u[a]

    u_a_grad = (np.dot(u_a.T,v_i)+b_u[a]+b_v[i]-Y_ai)*v_i + lam_u[a] * u_a
    b_u_grad = (np.dot(u_a.T,v_i)+b_u[a]+b_v[i]-Y_ai)

    v_i_grad = (np.dot(u_a.T,v_i)+b_u[a]+b_v[i]-Y_ai)*u_a + lam_v[i] * v_i
    b_v_grad = (np.dot(u_a.T,v_i)+b_u[a]+b_v[i]-Y_ai)
    u[a] = u[a] - step* u_a_grad
    b_u[a] = b_u[a] - step* b_u_grad
    v[i] = v[i] - step* v_i_grad
    b_v[i] = b_v[i] - step* b_v_grad
    x = (u,b_u,v,b_v)
    return x

In [75]:
def sgd_step_test():
  '''
  This is a test function provided to help you debug your implementation
  '''
  step = 0.025
  lam =(np.array([ 0.00333333,  0.005,  0.00333333,  0.005,  0.00333333]), np.array([ 0.0025,  0.00333333,  0.01,  0.002]))
  
  np.random.seed(0)
  
  first = []
  for i in range(5):
    first.append(np.random.rand(2, 1))
  second = np.zeros((5, 1))
  third = []
  for i in range(5):
    third.append(np.random.rand(2, 1))
  fourth = np.zeros((5, 1))
  x0 = (first, second, third, fourth)
  
  x_result = sgd_step(ratings_small[3], x0, lam, step)
  
  x_list = [[u.tolist() for u in x_result[0]],
            x_result[1].tolist(),
            [v.tolist() for v in x_result[2]],
            x_result[3].tolist()]
    
  assert np.all(np.isclose(x_list[0], np.array([[[0.5488135039273248], [0.7151893663724195]],
                                                [[0.6667107015911342], [0.5875840438721468]],
                                                [[0.4236547993389047], [0.6458941130666561]],
                                                [[0.4375872112626925], [0.8917730007820798]],
                                                [[0.9636627605010293], [0.3834415188257777]]])))
  assert np.all(np.isclose(x_list[1], np.array([[0.0], [0.08086477989447478], [0.0], [0.0], [0.0]])))
  assert np.all(np.isclose(x_list[2], np.array([[[0.8404178830022684], [0.5729237224816648]],
                                                [[0.5680445610939323], [0.925596638292661]],
                                                [[0.07103605819788694], [0.08712929970154071]],
                                                [[0.02021839744032572], [0.832619845547938]],
                                                [[0.7781567509498505], [0.8700121482468192]]])))
  assert np.all(np.isclose(x_list[3], np.array([[0.08086477989447478], [0.0], [0.0], [0.0], [0.0]])))
  print("Test passed!")
sgd_step_test()

mf_sgd(ratings_small, ratings_small, step_size_fn=lambda i: 0.1,
       lam=0.01, max_iter=1000, k=2)

Test passed!
SGD result for k = 2 : rmse train = [[0.00464191]] ; rmse validate = [[0.00464191]]


([array([[2.01544792],
         [0.3561021 ]]),
  array([[1.42507837],
         [0.48798664]]),
  array([[-1.48635907],
         [ 0.07673984]]),
  array([[-0.13158296],
         [-0.8980307 ]]),
  array([[ 2.27448457],
         [-1.34763427]])],
 array([0.36069283, 0.19597829, 1.96549433, 0.44934677, 1.24485534]),
 [array([[1.55024696],
         [0.60202987]]),
  array([[0.88126647],
         [1.82086226]]),
  array([[1.67896045],
         [0.37812022]]),
  array([[-0.57784711],
         [-1.32652495]])],
 array([1.29609553, 0.2052761 , 0.43746584, 2.27753008]))

## 4) MovieLens

Movie ratings compution using ALS 
dataset from https://grouplens.org/datasets/movielens/

In [88]:
# After retrieving the output x from mf_als, you can use this function to save the output so
# you don't have to re-train your model
def save_model(x):
    pickle.dump(x, open("ALSmodel", "wb"))

# After training and saving your model once, you can use this function to retrieve the previous model
def load_model():
    x = pickle.load(open("ALSmodel", "rb"))
    return x

# Compute the root mean square error
def rmse(data, x):
    error = 0.
    for datum in data:
        error += (datum[-1] - pred(datum, x))**2
    return np.sqrt(error/len(data))

# Counts of users and movies, used to calibrate lambda
def counts(data, index):
    item_count = {}
    for datum in data:
        j = datum[index]
        if j in item_count:
            item_count[j] += 1
        else:
            item_count[j] = 1
    c = np.ones(max(item_count.keys())+1)
    for i,v in item_count.items(): c[i]=v
    return c

# The ALS outer loop
def mf_als(data_train, data_validate, k=2, lam=0.02, max_iter=100, verbose=False):
    # size of the problem
    n = max(d[0] for d in data_train)+1 # users
    m = max(d[1] for d in data_train)+1 # items
    # which entries are set in each row and column
    us_from_v = [[] for i in range(m)]
    vs_from_u = [[] for a in range(n)]
    for (a, i, r) in data_train:
        us_from_v[i].append((a, r))
        vs_from_u[a].append((i, r))
    # Initial guess at u, b_u, v, b_v
    # Note that u and v are lists of column vectors (columns of U, V).
    x = ([np.random.normal(1/k, size=(k,1)) for a in range(n)],
          np.zeros(n),
          [np.random.normal(1/k, size=(k,1)) for i in range(m)],
          np.zeros(m))
    # Alternation, modifies the contents of x
    start_time = time.time()
    for i in range(max_iter):
        update_U(data_train, vs_from_u, x, k, lam)
        update_V(data_train, us_from_v, x, k, lam)
        if verbose:
            print('train rmse', rmse(data_train, x), 'validate rmse', data_validate and rmse(data_validate, x))
        if data_validate == None: # code is slower, print out progress
            print("Iteration {} finished. Total Elapsed Time: {:.2f}".format(i + 1, time.time() - start_time))
    # The root mean square errors measured on validate set
    if data_validate != None:
        print('ALS result for k =', k, ': rmse train =', rmse(data_train, x), '; rmse validate =', rmse(data_validate, x))
    return x

# Simple validate case
mf_als(ratings_small, ratings_small,lam=0.01, max_iter=10, k=2)

ALS result for k = 2 : rmse train = [[0.00672852]] ; rmse validate = [[0.00672852]]


([array([[-0.18386787],
         [-2.9655237 ]]),
  array([[ 0.7045643 ],
         [-1.93463287]]),
  array([[-1.64317857],
         [ 2.22671523]]),
  array([[-0.67642905],
         [ 1.85737747]]),
  array([[-0.56865292],
         [ 2.20820726]])],
 array([2.59866461, 1.57913883, 4.71547332, 3.37654252, 3.11917172]),
 [array([[ 1.040202  ],
         [-0.88537615]]),
  array([[ 1.45491916],
         [-0.38127039]]),
  array([[0.],
         [0.]]),
  array([[0.54783477],
         [0.5177597 ]])],
 array([-0.03027794, -0.4625075 ,  1.88082828,  0.03744944]))

In [89]:
## provided code

def load_ratings_data_small(path_data='ratings.csv'):
    """
    Returns two lists of triples (a, i, r) (training, validate)
    """
    # we want to "randomly" sample but make it deterministic
    def user_hash(uid):
        return 71 * uid % 401
    def user_movie_hash(uid, iid):
        return (17 * uid + 43 * iid) % 61
    data_train = []
    data_validate = []
    with open(path_data) as f_data:
        for line in f_data:
            (uid, iid, rating, timestamp) = line.strip().split(",")
            h1 = user_hash(int(uid))
            if h1 <= 40:
                h2 = user_movie_hash(int(uid), int(iid))
                if h2 <= 12:
                    data_validate.append([int(uid), int(iid), float(rating)])
                else:
                    data_train.append([int(uid), int(iid), float(rating)])
    print('Loading from', path_data,
          'users_train', len(set(x[0] for x in data_train)),
          'items_train', len(set(x[1] for x in data_train)),
          'users_validate', len(set(x[0] for x in data_validate)),
          'items_validate', len(set(x[1] for x in data_validate)))
    return data_train, data_validate

def load_ratings_data(path_data='ratings.csv'):
    """
    Returns a list of triples (a, i, r)
    """
    data = []
    with open(path_data) as f_data:
        for line in f_data:
            (uid, iid, rating, timestamp) = line.strip().split(",")
            data.append([int(uid), int(iid), float(rating)])

    print('Loading from', path_data,
          'users', len(set(x[0] for x in data)),
          'items', len(set(x[1] for x in data)))
    return data

def load_movies(path_movies='movies.csv'):
    """
    Returns a dictionary mapping item_id to item_name and another dictionary
    mapping item_id to a list of genres
    """
    data = {}
    genreMap = {}
    with open(path_movies, encoding = "utf8") as f_data:
        for line in f_data:
            parts = line.strip().split(",")
            item_id = int(parts[0])
            item_name = ",".join(parts[1:-1]) # file is poorly formatted
            item_genres = parts[-1].split("|")
            data[item_id] = item_name
            genreMap[item_id] = item_genres
    return data, genreMap

def baseline(train, validate):
    item_sum = {}
    item_count = {}
    total = 0
    for (i, j, r) in train:
        total += r
        if j in item_sum:
            item_sum[j] += 3
            item_count[j] += 1
        else:
            item_sum[j] = r
            item_count[j] = 1
    error = 0
    avg = total/len(train)
    for (i, j, r) in validate:
        pred = item_sum[j]/item_count[j] if j in item_count else avg
        error += (r - pred)**2
    return np.sqrt(error/len(validate))

# Load the movie data
# Below is code for the smaller dataset, used in section 3 of the HW
def tuning_als(max_iter_als=20, verbose=False):
    b1, v1 = load_ratings_data_small()
    print('Baseline rmse (predict item average)', baseline(b1, v1))
    print('Running on the MovieLens data')
    lams = [0.1,1,10,100]
    ks = [1,2,3]
    for k in ks:
        for lam in lams:
            print('ALS, k =', k, 'and lam', lam)
            mf_als(b1, v1, lam = lam, max_iter=max_iter_als, k=k, verbose=verbose)

### begin my code

#### 4.1) Recommendations

In [91]:
# No need to run a second time - use saved model instead (see next cell)

data = load_ratings_data()
movies_dict, genres_dict = load_movies()
model = mf_als(data, None, k=10, lam=1, max_iter=20)
save_model(model)

Loading from ratings.csv users 13366 items 2000
Iteration 1 finished. Total Elapsed Time: 12.01
Iteration 2 finished. Total Elapsed Time: 23.89
Iteration 3 finished. Total Elapsed Time: 35.82
Iteration 4 finished. Total Elapsed Time: 47.74
Iteration 5 finished. Total Elapsed Time: 59.90
Iteration 6 finished. Total Elapsed Time: 71.75
Iteration 7 finished. Total Elapsed Time: 83.65
Iteration 8 finished. Total Elapsed Time: 95.56
Iteration 9 finished. Total Elapsed Time: 107.51
Iteration 10 finished. Total Elapsed Time: 119.65
Iteration 11 finished. Total Elapsed Time: 132.25
Iteration 12 finished. Total Elapsed Time: 144.86
Iteration 13 finished. Total Elapsed Time: 157.09
Iteration 14 finished. Total Elapsed Time: 168.85
Iteration 15 finished. Total Elapsed Time: 180.56
Iteration 16 finished. Total Elapsed Time: 192.56
Iteration 17 finished. Total Elapsed Time: 205.25
Iteration 18 finished. Total Elapsed Time: 217.12
Iteration 19 finished. Total Elapsed Time: 229.02
Iteration 20 finish

In [92]:
model = load_model()

In [163]:
### homework code to see the user data.  I should use pandas instead

def get_user_data(data, user):
    user_data = []
    for d in data:
        if d[0] == user:
            user_data.append(d)
    return user_data

def display_user_data(user_data,movies_dict,genres_dict, min_rating, genre_filter='None'):
    if genre_filter == None:
        use_filter = False
    else:
        use_filter = True
    print("Movie# \tRating\tGenres")

    
    for d in user_data:
        (user, movie, rating) = d
        genres=genres_dict[movie]
        movie = movies_dict[movie]
        if use_filter == True:
            if genre_filter not in genres:
                pass
            elif rating >= min_rating:
                print(f"{movie}\n{rating}\t{genres}\n")

## 4.1) User Data

By running the code below, we can see the list of films that the user has rated 5, and the genre.


In [94]:
guy = get_user_data(data, 270894)
print(guy)
display_user_data(guy,movies_dict,genres_dict, 5)

[[270894, 1148, 5.0], [270894, 661, 5.0], [270894, 2096, 5.0], [270894, 32031, 5.0], [270894, 4006, 5.0], [270894, 71264, 5.0], [270894, 2018, 5.0], [270894, 56152, 5.0], [270894, 8907, 5.0], [270894, 112175, 5.0], [270894, 2700, 5.0], [270894, 673, 5.0], [270894, 97913, 5.0], [270894, 8961, 5.0], [270894, 4873, 5.0], [270894, 3034, 5.0], [270894, 5690, 5.0], [270894, 8965, 5.0], [270894, 2141, 5.0], [270894, 2090, 5.0], [270894, 595, 5.0], [270894, 594, 5.0], [270894, 50872, 5.0], [270894, 59784, 5.0], [270894, 45517, 5.0], [270894, 2987, 5.0], [270894, 27660, 5.0], [270894, 2078, 5.0], [270894, 81564, 5.0], [270894, 364, 5.0], [270894, 27904, 5.0], [270894, 3159, 5.0], [270894, 134853, 5.0], [270894, 37729, 5.0], [270894, 5971, 5.0], [270894, 103335, 5.0], [270894, 8360, 5.0], [270894, 3751, 5.0], [270894, 3000, 5.0], [270894, 1907, 5.0], [270894, 709, 5.0], [270894, 6283, 5.0], [270894, 1032, 5.0], [270894, 1030, 5.0], [270894, 2687, 5.0], [270894, 72226, 5.0], [270894, 60069, 5.0],

The code below is for predicting ratings for each of the movies he hasn't seen yet.

In [157]:
def eval_all_movies(user_data,model, movies_dict):
    (u, b_u, v, b_v) = model
    movies_seen = []
    recommended_movies = []
    movie_list = np.array([[],[]])
    for d in user_data:
        (user, movie, rating) = d
        movies_seen.append(movie)
    print("Movies seen: ",len(movies_seen),"\n")
    print("Number of total movies:", len(movies_dict), "\n")
    print("Calculating predicted ratings: \n")
    for i in movies_dict:
        if i not in movies_seen:
            predicted = pred((user,i,1),model)[0,0]
            #print(predicted)
            movie_list = np.hstack((movie_list,np.array([[i],[predicted]])))
    return movie_list

len(model)
movie_list = eval_all_movies(guy,model,movies_dict)

Movies seen:  244 

Number of total movies: 2000 

Calculating predicted ratings: 



The code here is for sorting and selecting the top 50 recommended films.  I know this is slow to literally make a list of all the movies, sort it, and then truncate it.

I think the ideal solution would probably be to implement a stack or something instead.

In [165]:
sorted_indexes = np.argsort(movie_list[1:],axis=1)
top_50 = np.squeeze(sorted_indexes[0][-50:])
new_list = movie_list[:,top_50]
""" sanity check debugging
first_val = movie_list[1,sorted_indexes[0][0]]
second_val = movie_list[1,sorted_indexes[0][-1]]
print("is ", first_val, " greater than ", second_val, "?")
"""
#print(new_list[0,-1],new_list[1,-1])
#print(len(new_list),"out of",len(genres_dict))
suggested_data = []
for counter in range(50):
    suggested_data.append((270894,new_list[0,-counter-1],new_list[1,-counter-1]))
    #print(new_list[1,-counter-1])


Display the top films, filtered by his favorite genre!

In [164]:
display_user_data(suggested_data,movies_dict,genres_dict, 1, genre_filter='Animation')

Movie# 	Rating	Genres
Wallace & Gromit: A Close Shave (1995)
5.398315526225244	['Animation', 'Children', 'Comedy']

Finding Nemo (2003)
5.3494063110595995	['Adventure', 'Animation', 'Children', 'Comedy']

Up (2009)
5.324664682720811	['Adventure', 'Animation', 'Children', 'Drama']

Toy Story 3 (2010)
5.236669811580798	['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'IMAX']

Big Hero 6 (2014)
5.226843319560599	['Action', 'Animation', 'Comedy']

"Monsters, Inc. (2001)"
5.1960174762482545	['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']

"Grand Day Out with Wallace and Gromit, A (1989)"
5.15650670551228	['Adventure', 'Animation', 'Children', 'Comedy', 'Sci-Fi']

Wallace & Gromit: The Best of Aardman Animation (1996)
5.12986858656058	['Adventure', 'Animation', 'Comedy']

Aladdin (1992)
5.112769819573992	['Adventure', 'Animation', 'Children', 'Comedy', 'Musical']

Howl's Moving Castle (Hauru no ugoku shiro) (2004)
5.025363624554047	['Adventure', 'Animation', 'Fantasy',

## 4.2 Similarity

Finding the closest films to each other based on their Vi vectors in the model.

We do this by computing CosSim = dot(v_a,v_b) / (norm(v_a)*norm(v_b)) - this will be proportional to the cosine between the two vectors, meaning that pairs of vectors with bigger CosSim magnitudes will be more similar

In [186]:
import math
def magnitude(vector): #taken from google
    return math.sqrt(sum(pow(element,2) for element in vector))

def CosSim(base_movie, movies_dict, model, n):
    (u, b_u, v, b_v) = model
    v_a = v[base_movie]
    print(v_a.shape)
    all_cs = np.array([[],[]]) #yes, I know this is lazy.  I have lots of memory right now.
    
    for movie in movies_dict:
        if movie != base_movie:
            v_b = v[movie]
            cs = np.dot(v_a.T,v_b) / (magnitude(v_a) * magnitude(v_b))
            cs = cs[0,0]
            all_cs = np.hstack((all_cs,np.array([[movie], [cs]])))
    indexes = np.argsort(all_cs[1,:],axis=0)
    top_n = all_cs[0,indexes[-n:]]
    return top_n


In [189]:
NewHopeID = 260
PhantomID = 2628
similar_list = CosSim(NewHopeID,movies_dict,model,10)
print(similar_list)
for movie in similar_list:
    print(movies_dict[movie])
similar_phantom = CosSim(PhantomID,movies_dict,model,10)
print(similar_phantom)
for movie in similar_phantom:
    print(movies_dict[movie])

(10, 1)
[   329.   3471.   1127.   2115.   2628. 122886.   1198.   1291.   1210.
   1196.]
Star Trek: Generations (1994)
Close Encounters of the Third Kind (1977)
"Abyss, The (1989)"
Indiana Jones and the Temple of Doom (1984)
Star Wars: Episode I - The Phantom Menace (1999)
Star Wars: Episode VII - The Force Awakens (2015)
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
Indiana Jones and the Last Crusade (1989)
Star Wars: Episode VI - Return of the Jedi (1983)
Star Wars: Episode V - The Empire Strikes Back (1980)
(10, 1)
[ 3615.   260.  1196. 51662. 59615.  6934.  1210.  6365.  5378. 33493.]
Dinosaur (2000)
Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode V - The Empire Strikes Back (1980)
300 (2007)
Indiana Jones and the Kingdom of the Crystal Skull (2008)
"Matrix Revolutions, The (2003)"
Star Wars: Episode VI - Return of the Jedi (1983)
"Matrix Reloaded, The (2003)"
Star Wars: Episode II - Attack of the Clones (2002)
Star Wars: Episode III 

### 4.2B)

The following movies are most similar to A New Hope:

[   329.   3471.   1127.   2115.   2628. 122886.   1198.   1291.   1210.
   1196.]
Star Trek: Generations (1994)

Close Encounters of the Third Kind (1977)

"Abyss, The (1989)"

Indiana Jones and the Temple of Doom (1984)

Star Wars: Episode I - The Phantom Menace (1999)

Star Wars: Episode VII - The Force Awakens (2015)

Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)

Indiana Jones and the Last Crusade (1989)

Star Wars: Episode VI - Return of the Jedi (1983)

Star Wars: Episode V - The Empire Strikes Back (1980)

### 4.2C)

For Phantom Menace:
[ 3615.   260.  1196. 51662. 59615.  6934.  1210.  6365.  5378. 33493.]

Dinosaur (2000)

Star Wars: Episode IV - A New Hope (1977)

Star Wars: Episode V - The Empire Strikes Back (1980)

300 (2007)

Indiana Jones and the Kingdom of the Crystal Skull (2008)

"Matrix Revolutions, The (2003)"

Star Wars: Episode VI - Return of the Jedi (1983)

"Matrix Reloaded, The (2003)"

Star Wars: Episode II - Attack of the Clones (2002)

Star Wars: Episode III - Revenge of the Sith (2005)

### 4.2D-G

Now we look for broader similarities between and within genres

In [214]:
def Similarity(movie1, movie2, model):
    (u, b_u, v, b_v) = model
    v_a = v[movie1]
    v_b = v[movie2]
    return np.dot(v_a.T,v_b) / (magnitude(v_a) * magnitude(v_b))

key_view = movies_dict.keys()
moviekeys = []
for movie in key_view:
    moviekeys.append(movie)

running_sum = 0
running_count = 0
for i in range(len(movies_dict)-1):
    for j in range(len(movies_dict)-i-1):
        running_sum = running_sum + Similarity(moviekeys[i],moviekeys[i+j+1], model)
        running_count +=1
    if i in [200,450,750,1100]: # 900x900 pairs left, 1250x1250 pairs left, 1550x1550 pairs left, 1800x1800 pairs left.  roughly every 20% completion
        print("**")
print(f"Average similarity between unique movie pairs: {running_sum/running_count}")

**
**
**
**
Average similarity between unique movie pairs: [[0.0077457]]


In [212]:
len(movies_dict)
range(len(movies_dict)-1998)

Average similarity between unique movie pairs: [[0.0077457]]


In [215]:
#i'm sure there's an elegant way to do for (movie, movie) in (movies, movies)
#but i'm doing it the simple way that I know

#i'll compare my solution with the hw solution afterwards

similarity_dict = {}
for genre in genres: #19 genres
    similarity_dict[genre] = np.zeros(2) # one column for sum and one column for count
n = len(movies_dict) 
for i in range(n-1): #from 0 to 1998
    genres_i = genres_dict[moviekeys[i]] #get genres of i'th movie
    for j in range(n-i-1): #from 1999 to 1
        genres_j = genres_dict[moviekeys[i+j+1]] # get genre of j'th movie after i
        for g in genres_i:
            if g in genres_j:
                similarity_dict[g][0] = similarity_dict[g][0] + Similarity(moviekeys[i],moviekeys[i+j+1],model)
                similarity_dict[g][1] = similarity_dict[g][1] + 1
    if i in [200,450,750,1100]: # 900x900 pairs left, 1250x1250 pairs left, 1550x1550 pairs left, 1800x1800 pairs left.  roughly every 20% completion
        print("**")

""" Class code for reference:
for genre in genres:
    movies = [i for i, my_genres in genres_dict.items() if genre in my_genres]   #this part is something i should look at.  looks like it works like a SQL selection, so you're not comparing all movies to all movies
    sims = []
    for i in movies:
        for j in movies:
            if i == j: continue
            sim = v[i].T.dot(v[j])/(np.linalg.norm(v[i])*np.linalg.norm(v[j]))
            sims.append(sim)
    print(genre, np.mean(sims))
""" #not too different from my code

2000
**
**
**
**


In [220]:
for item in similarity_dict:
    print(item, "\tAverage similarity\t", similarity_dict[item][0]/similarity_dict[item][1], "\n")

Western 	Average similarity	 0.10016602690374397 

Comedy 	Average similarity	 0.07559470723027109 

Children 	Average similarity	 0.27229484186459524 

Crime 	Average similarity	 0.06696085357633019 

Musical 	Average similarity	 0.27501819151074625 

Adventure 	Average similarity	 0.07135447165040337 

Drama 	Average similarity	 0.06768486911697487 

Horror 	Average similarity	 0.2492613777469407 

War 	Average similarity	 0.09797277183531476 

Documentary 	Average similarity	 0.33119984785774526 

Romance 	Average similarity	 0.08462725574907316 

Animation 	Average similarity	 0.3071465761929819 

Film-Noir 	Average similarity	 0.4019363901421902 

Sci-Fi 	Average similarity	 0.14538503096293928 

Mystery 	Average similarity	 0.06376430882883291 

Fantasy 	Average similarity	 0.09753358168777176 

IMAX 	Average similarity	 0.3276036130440015 

Action 	Average similarity	 0.11896151409934627 

Thriller 	Average similarity	 0.0824171365560819 



We see that film-noirs are the most similar to each other, and mysteries the least similar.
The autograder accepted these answers as within tolerance but had slightly different similarity scores (and chose Crime as least similar, not Mystery)

In [230]:
comedy_dict = {}
_, _, v, _ = model
for genre in genres:
    comedies = [i for i, my_genres in genres_dict.items() if "Comedy" in my_genres]
for genre in genres:
    movies = [i for i, my_genres in genres_dict.items() if genre in my_genres]
    sims = []
    for i in comedies:
        for j in movies:
            if i == j: continue
            if j in comedies: continue
            sim = v[i].T.dot(v[j])/(np.linalg.norm(v[i])*np.linalg.norm(v[j]))
            sims.append(sim)
    print(genre, np.mean(sims))

Western -0.06542239168968507
Comedy nan
Children 0.052622799640206684
Crime -0.04343356330561578
Musical 0.031184848280891242
Adventure -0.025442389830672246
Drama -0.029766456172157597
Horror -0.05380309653999443
War -0.07877965598549858
Documentary 0.05318609130271492
Romance 0.003690483753478649
Animation 0.04886051784040335
Film-Noir -0.05521928140636941
Sci-Fi -0.04034487783480902
Mystery -0.042793202368592465
Fantasy -0.007804748292963473
IMAX -0.04512889018118615
Action -0.039017135410493196
Thriller -0.037633040928335894


Children category is most similar, and War category is least similar.

## Optional HW: Collaborative filtering and the SVD
### Eigen Vector and the Half-Blood Python

Singular value decompositions: decomposing a large (mostly empty) matrix Y into the product of
n x k matrix U and k x m matrix V.T

Using black magic to construct orthogonal bases for U and V, we can then write U and V.T as a product of
three matrices that are each composed of eigen-stuff.

We will derive U and V by having Python calculate the eigen-stuff for a small matrix Y, then compose U and V.

In [271]:
Y = np.array([[4,3,1],[1,3,-2],[5,2,3]])
(r_eval, r_evec) = np.linalg.eig(Y.T @ Y)	# gives right eigenvalues and eigenvectors
(l_eval, l_evec) = np.linalg.eig((Y @ Y.T).T)	# gives left eigenvalues and eigenvectors

In [272]:
((r_eval, r_evec),\
(l_eval, l_evec))

r_eval = r_eval[0::2] #drop the middle eigenvalue
r_evec = r_evec[0::2,:]
l_eval = l_eval[0::2]
l_evec = l_evec[0::2,:]

We examine the eigenvalues and see that two of them are essentially zero, suggesting that we can take the rank 2 principal component of Y

In [274]:
Big_Lambda = np.array([[r_eval[0]**.5,0],[0,r_eval[1]**0.5]])

(   (r_eval, r_evec),\
    (l_eval, l_evec),\
    (r_norms, l_norms))


((array([63.97999199, 14.02000801]),
  array([[ 0.80844891,  0.57735027,  0.11435482],
         [ 0.30519027, -0.57735027,  0.75731471]])),
 (array([63.97999199, 14.02000801]),
  array([[ 0.6311939 ,  0.75180941, -0.19072721],
         [ 0.74565815, -0.52048344,  0.41604197]])),
 (array([[1.],
         [1.]]),
  array([[1.],
         [1.]])))

In [275]:
matrix_V = np.array([r_evec[0],r_evec[1]]).T
matrix_U = np.array([l_evec[0],l_evec[1]]).T



In [277]:
#print(matrix_U,matrix_V.T)
print(matrix_U@Big_Lambda@matrix_V.T)

[[ 4.93375427  1.30294768  2.69176554]
 [ 4.26686222  4.59709199 -0.78822497]
 [-0.75792788 -1.78018892  1.00528635]]


Honestly, this matrix bears only minor resemblance to the source matrix Y.  I'm not sure what to make of this.